## **Eduardo Carrasco Vidal** <img src="img/logo.png" align="right" style="width: 120px;"/>

**Magister en Inteligencia Artificial, Universidad Adolfo Ibáñez.**

**Profesor:** John Atkinson.
**Curso:** Procesamiento del Leguaje Natural (Natural Language Processing).

Enlace al repositorio del alumno en [GitHub](https://github.com/educarrascov/MIA_NaturalLP) _@educarrascov_

![Python](https://img.shields.io/badge/python-%2314354C.svg) 

**CLASIFICACIÓN DE SENTIMIENTOS CON LSA**

Este programa utiliza  *Análisis Semántico Latente*  (LSA) para generar [embeddings](https://es.wikipedia.org/wiki/Word_embedding) que permitan realizar posteriormente una actividad de clasificación de sentimientos.

La clasificación se realiza entrenando un modelo [Bayesiano Gausiano](https://iq.opengenus.org/gaussian-naive-bayes/).

Primero, necesitamos instalar algunos paquetes:

In [3]:
# pip install es_core_news_sm

In [ ]:
import es_core_news_sm
import os
import pandas as pd
import regex
import numpy as np
from scipy.spatial.distance import cosine
from spacy.lang.es.stop_words import STOP_WORDS
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import matplotlib.pyplot as plt
from numpy.linalg import svd

Definamos la función **CrearEspacioLSA(corpus,dim,NombreModelo)**, que  utiliza LSA para crear un *espacio semántico* de dimensiones reducidas (i.e., modelo vectorial que representa documentos y palabras), que se graba y luego puede ser cargado por otros programas. La función recibe la lista de documentos pre-procesados (**corpus**), el número de dimensiones a reducir (**dim**) vía [SVD](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491), y el nombre de la carpeta donde se grabará el modelo (**NombreModelo**).

LSA utiliza la *descomposición de valores singulares* (SVD) para descomponer la matriz de frecuencias original vía **TfidfVectorizer** en tres matrices: $U$, $\Sigma$ y $V^T$. Luego, las nuevas representaciones vectoriales en dimensiones reducidas (**dim**)  se reconstruyen  como:

*   Representación de  términos: $U(dim) *  \Sigma(dim)$

*   Representación  de documentos: $\Sigma(dim) * V^T(dim)$

In [1]:
def CrearModeloLSA(textos,dim,NombreModelo):
  MatrizFrec = TfidfVectorizer()
  tf = MatrizFrec.fit_transform(textos).T
  U, Sigma, VT = svd(tf.toarray())
  # Se realiza producto punto de matrices con las nuevas dimensiones
  terms = np.dot(U[:,:dim], np.diag(Sigma[:dim]))
  docs  = np.dot(np.diag(Sigma[:dim]), VT[:dim, :]).T 
  vocab = MatrizFrec.get_feature_names()
  GrabarModeloLSA(NombreModelo, Sigma, terms, docs, vocab)

def GrabarModeloLSA(NombreModelo,Sigma,terms,docs, vocab):
   existe = os.path.isdir(NombreModelo)
   if not existe:
       os.mkdir(NombreModelo)
   joblib.dump(Sigma,   NombreModelo +"/"+'sigma.pkl') 
   joblib.dump(terms,   NombreModelo +"/"+'terms.pkl') 
   joblib.dump(docs,    NombreModelo +"/"+'docs.pkl') 
   joblib.dump(vocab,   NombreModelo +"/"+'vocab.pkl') 

def CargarModeloLSA(NombreModelo):
    terms   = joblib.load(NombreModelo+"/"+'terms.pkl')
    vocab   = joblib.load(NombreModelo+"/"+'vocab.pkl')
    modelo =  CrearDiccionario(terms,vocab)
    return(modelo)
def CrearDiccionario(lista_vectores,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista_vectores[v]
   return(dicc)

Un aspecto importante al utilizar LSA es determinar cuál es el número óptimo de dimensiones a reducir. Esto depende del tamaño del corpus utiizado para construir el modelo:

1.   Si el tamaño es muy grande (i.e., varios GigaBytes de texto), lo adecuado es utilizar entre 200 a 300 dimensiones.
2.   Si el tamaño  es pequeño, se puede elegir el número de dimensiones como el número de valores singulares de la matriz $\Sigma$) que maximice la *importancia* (descartando el primer valor pues corelaciona con el largo del corpus). La importancia de un valor singular $x$ es simplemente $x^2$.

Para visualizar la importancia de los valores singulares, podemos definir  la función **GraficarImportancia($\Sigma$)**, que grafica los valores singulares (i.e., dimensiones) versus importancia:

In [2]:
def GraficarImportancia(Sigma):   
    NumValores = np.arange(len(Sigma))
    Importancia = [x**2 for x in Sigma]
    plt.bar(NumValores,Importancia)
    plt.ylabel('Importancia')
    plt.xlabel('Valores Singulares')
    plt.title('Importancia de Valores Singulares en SVD')
    plt.show()

Luego, definimos algunas funciones utilitarias para "limpieza" y pre-procesamieto:

In [3]:
 def PreProcesar(textos):
    texto_limpio = []
    for texto in textos:  
        texto = Lematizar(texto)     
        texto = EliminaNumeroYPuntuacion(texto)      
        texto_limpio.append(texto)
    return(texto_limpio)

def Lematizar(oracion):
   doc = nlp(oracion)
   lemas = [token.lemma_ for token in doc]
   return(" ".join(lemas))  

def EliminaNumeroYPuntuacion(oracion):
    string_numeros = regex.sub(r'[\”\“\¿\°\d+]','', oracion)
    return ''.join(c for c in string_numeros if c not in punctuation)

def Tokenizar(oracion):
    doc = nlp(oracion)
    tokens = [palabra.text for palabra in doc]
    return(tokens)

def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for NombreArchivo  in directorio:
     try:
          texto = open(path+NombreArchivo,'r',encoding="utf-8").read()
          corpus.append(texto)
          doc_id.append(NombreArchivo)
     except IsADirectoryError:
          texto = ""
  return(corpus,doc_id)

Una vez que generamos las nuevas representaciones vectoriales para documentos y términos, podríamos utilizar dichos  vectores para realizar diversas tareas tales como clustering, clasificación, comparación de documentos, etc.

Como ejemplo, definamos la función **GraficarVectores(vocab,vectores)**, que toma vectores de documentos (o palabras), y el vocabulario, y grafica cada uno de los elementos en un gráfico 2-dimensional. Dado que los vectores contienen más de 2 dimensiones, en este ejemplo, sólo tomamos las 2 primeras. 

El gráfico le permitirá visualizar espacialmente la *cercanía* que existe entre documentos para posteriores análisis.

Alternativamente, Ud. podría aplicar otros métodos de reducción dimensional para graficas las mejores 2 componentes:

In [4]:
def GraficarVectores(vocab,vectores):
    x = []
    y = []
    for value in vectores:
        x.append(value[0])
        y.append(value[1])   
    plt.figure(figsize=(7, 7))   
    plt.title("Distribución Espacial de Documentos")
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(vocab[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

Note que los vectores para documentos o palabras están indexados por posición y no por nombre, lo que podría dificultad el acceso. Para mejorar esto, podemos definir la función **CrearDiccionario(Vectores,vocabulario)**, que dado un conjunto de **vectores** (de documentos o palabras) y un **vocabulario**, genera un nuevo arreglo donde el índice es el nombre correspondiente: una palabra en el caso de vectores de palabras o un nombre de documento en el caso de vectores de documentos:

In [5]:
def ObtenerEmbeddingOracion(modelo, dim, oracion):
   Lista_enbeddings = []
   Tokens = Tokenizar(oracion)
   for w in Tokens:
       # Verificar que la palabra w exista en el modelo
       try:
           modelo[w]
       except KeyError:
           continue
       # Obtener vector de la palabra w
       embedding = modelo[w]
       Lista_enbeddings.append(embedding)
   embedding_palabras = np.array(Lista_enbeddings)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(dim)
   return(embedding_oracion) 

Ahora, realizamos nuestro programa principal donde ajustamos la ruta donde se encuentran los documentos del corpus e inicializamos los modelos de lenguaje:

In [6]:
PATH = "DiscursosOriginales/"
nlp = es_core_news_sm.load()

NameError: name 'es_core_news_sm' is not defined

Creamos un corpus a partir de los documentos de la carpeta de *PATH*. Luego, creamos el espacio semántico utilizando LSA y lo grabamos para recuperarlo posteriormente. Por ahora podemos suponer que vamos a reducir el espacio original a $dim=3$ ($dim$ debe ser menor que el número de documentos): 

In [258]:
dim =  200
if os.path.isdir("mi_lsa"):
    modeloLSA = CargarModeloLSA("mi_lsa")
else:
    corpus, lista_docs = CrearCorpus(PATH+'Train/')
    textos  = PreProcesar(corpus)
    CrearModeloLSA(textos,dim,"mi_lsa")
    modeloLSA = CargarModeloLSA("mi_lsa")

In [260]:
textos_test = os.listdir(PATH+'Test/')
textos_test

['100178.txt',
 '100227.txt',
 '100454.txt',
 '101255.txt',
 '101415.txt',
 '101614.txt',
 '101766.txt',
 '102005.txt',
 '102351.txt',
 '102791.txt',
 '103122.txt',
 '103313.txt',
 '103511.txt',
 '134150.txt',
 '134658.txt',
 '134801.txt',
 '135653.txt',
 '135757.txt',
 '135814.txt',
 '136230.txt',
 '136280.txt',
 '136377.txt',
 '136426.txt',
 '137702.txt',
 '138176.txt',
 '148292.txt',
 '148522.txt',
 '148806.txt',
 '149988.txt',
 '150322.txt',
 '150341.txt',
 '150633.txt',
 '151570.txt',
 '151937.txt',
 '152081.txt',
 '152091.txt',
 '152168.txt',
 '152358.txt',
 '152471.txt',
 '152616.txt',
 '153144.txt',
 '153186.txt',
 '165020.txt',
 '165487.txt',
 '165648.txt',
 '71541.txt',
 '71862.txt',
 '72017.txt',
 '72231.txt',
 '72656.txt',
 '72692.txt',
 '73117.txt',
 '73202.txt',
 '73235.txt',
 '73260.txt',
 '73601.txt',
 '73739.txt',
 '73742.txt',
 '73766.txt',
 '74214.txt',
 '74598.txt',
 '75121.txt',
 '75615.txt',
 '76138.txt',
 '76163.txt',
 '76467.txt',
 '76480.txt',
 '76776.txt',
 '7

In [234]:
texto = open(PATH+'Test/76163.txt','r',encoding="utf-8").read()

In [235]:
print(texto)

Muy buenos días:

 

A veces, los políticos nos enfrascamos en grandes discusiones conceptuales e ideológicas y, otras veces, nos olvidamos de la vida cotidiana de las personas.

 

Y hoy día, vamos a hablar de algo que afecta directamente la vida cotidiana de millones de chilenos, porque esta iniciativa de “Chile Sin Barreras”, que significa reemplazar los peajes físicos por peajes virtuales, electrónicos o digitales, le va a cambiar para mejor la vida a millones y millones de chilenos.

 

El ministro ya lo dijo, este nuevo sistema de “Chile Sin Barreras”, significa, en primer lugar, ahorrar tiempo: 4 millones de horas/hombre con estas iniciativas; 15 millones de horas/hombre cuando lo extendamos a todo el país. Y ese mayor tiempo lo vamos a dedicar a lo que realmente importa: a la familia, a los amigos, a la cultura, al deporte, a la recreación, a la reflexión y, también, a la oración.

 

Pero, además, va a significar un gran ahorro de recursos: 150 millones de dólares la primera e

In [275]:
def resumir_texto(texto, lineas=0):
    nlp_text=nlp(texto)
    oraciones=[]
    for sent in nlp_text.doc.sents:
        oraciones.append(str(sent).replace('\xa0','').replace('\n',''))
    oraciones_new=[EliminaNumeroYPuntuacion(Lematizar(oracion)) for oracion in oraciones]
    features = [ObtenerEmbeddingOracion(modeloLSA, dim, oracion) for oracion in oraciones_new]
    features = np.array(features)
    print(f"El texto contiene {len(features)} parrafos")
    if lineas==0 or lineas<0:
        resumen=int(len(features)/2)
    else:
        resumen=lineas
    print(f"Se escogerán {resumen} parrafos para el resumen")
    similitudes = []
    vector_promedio= features.mean(axis=0)
    for feature in features: 
        similitud = 1-cosine(vector_promedio,feature)
        similitudes.append(similitud)
    print(f"Las similitudes encontradas son \n" + '\n'.join([str(x) for x in similitudes]))
    df = pd.DataFrame({'oraciones':oraciones, 'oraciones_new':oraciones_new, 'similitudes':similitudes})
    resumen_texto = [x for x in df.sort_values(by='similitudes').head(11).sort_index()['oraciones']]
    return resumen_texto

In [276]:
resumen=resumir_texto(texto)

El texto contiene 23 parrafos
Se escogerán 11 parrafos para el resumen
Las similitudes encontradas son 
0.996876869875476
0.9968810640292879
0.9922616368932953
0.994984029951167
0.9946886588390521
0.9521718363663894
0.9931162512799386
0.9665074381381658
0.9991035926054099
0.9908520887640158
0.9970724762501603
0.9962173634581319
0.9972847078127947
0.9874634217477118
0.9965814391107779
0.9947077583232989
0.992775397861699
0.9679485324483886
0.9936642730141638
0.9950278012267574
0.999232119850694
0.9983147193726731
0.7495089705091262


In [277]:
print( '\n'.join(resumen))

El ministro ya lo dijo, este nuevo sistema de “Chile Sin Barreras”, significa, en primer lugar, ahorrar tiempo: 4 millones de horas/hombre con estas iniciativas; 15 millones de horas/hombre cuando lo extendamos a todo el país.
Pero, además, va a significar un gran ahorro de recursos: 150 millones de dólares la primera etapa; 600 millones de dólares cuando lo tengamos implementado en todas las carreteras del país.
Pero no es solamente tiempo y recursos.
El que los autos y los camiones no tengan que detenerse, también va a hacer que nuestras carreteras sean más seguras, porque en esas detencionesse producen muchos accidentes.
Y al ser carreteras más fluidas, van a ser carreteras más seguras y vamos también poder salvar vidas.
Estamos partiendo por los accesos a Santiago.
Yo creo que ésta es una buena noticia para todos nuestros compatriotas.
Pero nuestra preocupación como Gobierno es una sola: hacer que la vida sea más plena y más feliz para todos nuestros compatriotas.
Y para eso ¿se re